# Lab 5

Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [115]:
##### Part 1) Create the Matrix Class #####

class Matrix:
    def __init__(self, n=None, m=None, values=None):
        if values is not None:
            if not all(len(row) == len(values[0]) for row in values):
                raise ValueError("All rows must have the same number of columns")
            self.data = values
        elif n is not None and m is not None:
            self.data = [[0] * m for _ in range(n)]
        else:
            raise ValueError("Invalid initialization parameters")

# Part 2: Modified __getitem__ method.
    def __getitem__(self, indices):
        if isinstance(indices, tuple):
            i, j = indices
            if isinstance(i, slice) or isinstance(j, slice):
                return Matrix(values=[row[j] for row in self.data[i]])
            return self.data[i][j]
        return self.data[indices]

    def __setitem__(self, indices, value):
        if isinstance(indices, tuple):
            i, j = indices
            self.data[i][j] = value
        else:
            self.data[indices] = value

    def __repr__(self):
        return "\n".join(str(row) for row in self.data)

##### Part 2: Adding Additional Methods #####

    def shape(self):
        return (len(self.data), len(self.data[0]))
    
    def transpose(self):
        transposed = [[self.data[j][i] for j in range(len(self.data))] for i in range(len(self.data[0]))]
        return Matrix(values=transposed)
    
    def row(self, n):
        return Matrix(values=[self.data[n]])
    
    def column(self, n):
        return Matrix(values=[[row[n]] for row in self.data])
    
    def to_list(self):
        return self.data
    
    def block(self, n_0, n_1, m_0, m_1):
        return Matrix(values=[row[m_0:m_1] for row in self.data[n_0:n_1]])

##### Part 4: Adding Member Functions #####

    def scalarmul(self, c):
        return Matrix(values=[[c * value for value in row] for row in self.data])
    
    def add(self, N):
        if self.shape() != N.shape():
            print("Error: Matrices must have the same shape for addition.")
            return None
        return Matrix(values=[[self.data[i][j] + N.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])
    
    def sub(self, N):
        if self.shape() != N.shape():
            print("Error: Matrices must have the same shape for subtraction.")
            return None
        return Matrix(values=[[self.data[i][j] - N.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])
    
    def mat_mult(self, N):
        if len(self.data[0]) != len(N.data):
            print("Error: Matrix multiplication not possible with given dimensions.")
            return None
        return Matrix(values=[[sum(self.data[i][k] * N.data[k][j] for k in range(len(self.data[0]))) for j in range(len(N.data[0]))] for i in range(len(self.data))])
    
    def element_mult(self, N):
        if self.shape() != N.shape():
            print("Error: Matrices must have the same shape for element-wise multiplication.")
            return None
        return Matrix(values=[[self.data[i][j] * N.data[i][j] for j in range(len(self.data[0]))] for i in range(len(self.data))])
    
    def equals(self, N):
        return self.data == N.data
    
##### Part 5: Overloading Python Operators #####
    
    def __mul__(self, other):
        if isinstance(other, (int, float)):
            return self.scalarmul(other)
        elif isinstance(other, Matrix):
            return self.mat_mult(other)
        else:
            raise TypeError("Unsupported operand type for *: Matrix and {}".format(type(other)))
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def __add__(self, other):
        return self.add(other)
    
    def __sub__(self, other):
        return self.sub(other)
    
    def __eq__(self, other):
        return self.equals(other)
    
    def __assign__(self, other):
        if isinstance(other, Matrix):
            self.data = [row[:] for row in other.data]
        else:
            raise TypeError("Can only assign another Matrix instance.")

##### Testing Part 1 Functions #####

print("========== TESTING PART 1 FUNCTIONS ==========\n")

# Testing the implementation
print("Testing Matrix Initialization:")
m1 = Matrix(3, 3)
print("\nMatrix 1: Empty")
print(m1) # Expected Output: A 3x3 matrix of just 0s.

values = [[4, 1, 7], [3, 5, 9], [8, 3, 6]]
m2 = Matrix(values=values)
print("\nMatrix 2: Filled")
print(m2)  # Expected Output: A 3x3 matrix, counting from 1 to 9 by row.

print("\nMatrix 2: Testing Indexing")
print(m2[1, 2])  # Expected Output: Second Row, Third Column
print(m2[2][0])  # Expected Output: Third Row, First Column

print("\nMatrix 2: Edited 3rd Row, 2nd Column Value")
m2[2, 1] = 2
print(m2)
print(f"\nEdited value: {m2[2, 1]}")

##### Testing Part 2 Functions #####

print("\n========== TESTING PART 2 FUNCTIONS ==========")

print("\nShape of Matrix 2:")
print(m2.shape())  # Prints the amount of rows and columns.

print("\nTransposed Matrix 2:")
print(m2.transpose())  # Makes a transposed matrix.

print("\nRow 2 of Matrix 2:")
print(m2.row(1))  # Should return a matrix with the second row

print("\nColumn 3 of Matrix 2:")
print(m2.column(2))  # Should return a matrix with the second column

print("\nMatrix 2 as a List of Lists:")
print(m2.to_list())

print("\nSubmatrix only including columns 2 and 3, rows 1 and 2.")
print(m2.block(0, 2, 1, 3))  # Should return a submatrix

##### Testing Part 4 Functions #####

print("\n========== TESTING PART 4 FUNCTIONS ==========")

mA = Matrix(values=[[9, 2], [4, 5]])
mB = Matrix(values=[[6, 4], [1, 3]])

print(f"\nMatrix A: \n{mA}")
print(f"\nMatrix B: \n{mB}")
print(f"\nMatrix C: \n{m2}")

print("\nMultiplied Matrix B by 2:")
print(mB.scalarmul(2)) 

print("\nAdded Both Matrices:")
print(mA.add(mB))  

print("\nSubtracted Matrix B from Matrix A")
print(mA.sub(mB))  

print("\nMultiplied Both Matrices Together")
print(mA.mat_mult(mB))  

print("\nElement-Wise Multiplication of Both Matrices")
print(mA.element_mult(mB)) 

print("\nDoes matrix A = B?")
print(mA.equals(mB))  

print("\nDoes matrix B have the values [6, 4], [1, 3]?")
print(mB.equals(Matrix(values=[[6, 4], [1, 3]])))  

print("\n========== PART 4 ERROR HANDLING ==========")

print("\nAdded Matrix B with Matrix 2:")
print(mB.add(m2))

print("\nSubtract Matrix 2 from Matrix A:")
print(mA.sub(m2)) 

print("\nMultiplied Matrix B with Matrix 2:")
print(mB.mat_mult(m2)) 

print("\nElement-Wise Multiplication w/ Matrix A and Matrix 2:")
print(mB.element_mult(m2)) 

##### Testing Part 5 Operators #####

print("\n========== TESTING PART 5 OPERATORS ==========")

print("\nScalar Multiplication of Matrix A by 2:")
print(2 * mA)

print("\nAdding Matrixes A and B Together:")
print(mA + mB)

print("Subtracting Matrix B from A:")
print(mA - mB)

print("\nMultiplying Matrices A and B:")
print(mA * mB)

print("\nAre matrices A and B equal?")
print(mA == mB)

print("\nDoes Matrix A have the values [9,2], [4,5]?")
print(mA == Matrix(values=[[9, 2], [4, 5]]))

========== TESTING PART 1 FUNCTIONS ==========

Testing Matrix Initialization:

Matrix 1: Empty
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]

Matrix 2: Filled
[4, 1, 7]
[3, 5, 9]
[8, 3, 6]

Matrix 2: Testing Indexing
9
8

Matrix 2: Edited 3rd Row, 2nd Column Value
[4, 1, 7]
[3, 5, 9]
[8, 2, 6]

Edited value: 2

========== TESTING PART 2 FUNCTIONS ==========

Shape of Matrix 2:
(3, 3)

Transposed Matrix 2:
[4, 3, 8]
[1, 5, 2]
[7, 9, 6]

Row 2 of Matrix 2:
[3, 5, 9]

Column 3 of Matrix 2:
[7]
[9]
[6]

Matrix 2 as a List of Lists:
[[4, 1, 7], [3, 5, 9], [8, 2, 6]]

Submatrix only including columns 2 and 3, rows 1 and 2.
[1, 7]
[5, 9]

========== TESTING PART 4 FUNCTIONS ==========

Matrix A: 
[9, 2]
[4, 5]

Matrix B: 
[6, 4]
[1, 3]

Matrix C: 
[4, 1, 7]
[3, 5, 9]
[8, 2, 6]

Multiplied Matrix B by 2:
[12, 8]
[2, 6]

Added Both Matrices:
[15, 6]
[5, 8]

Subtracted Matrix B from Matrix A
[3, -2]
[3, 2]

Multiplied Both Matrices Together
[56, 42]
[29, 31]

Element-Wise Multiplication of Both Matrices
[54, 8

In [80]:
##### Part 3: Standalone Special Matrix Functions #####

def constant(n, m, c):
    return Matrix(values=[[float(c)] * m for _ in range(n)])

def zeros(n, m):
    return constant(n, m, 0.0)

def ones(n, m):
    return constant(n, m, 1.0)

def eye(n):
    return Matrix(values=[[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)])

##### Testing Part 3 Functions #####

print("========== TESTING PART 3 FUNCTIONS ==========\n")
print("Constant Matrix w/ 4.0:")
print(constant(4, 3, 4))  # Expected Output: 4x3 Matrix filled with 4.0.

print("Zero Matrix:")
print(zeros(2, 3))  # Expected Output: 2x3 matrix filled with 0.0.

print("Ones Matrix:")
print(ones(3, 2))  # Expected Output: 3x2 matrix filled with 1.0.

print("Identity Matrix:")
print(eye(3))  # Expected Output: 1.0s on the top left and bottom right diagonals.

========== TESTING PART 3 FUNCTIONS ==========

Constant Matrix w/ 4.0:
[4.0, 4.0, 4.0]
[4.0, 4.0, 4.0]
[4.0, 4.0, 4.0]
[4.0, 4.0, 4.0]
Zero Matrix:
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
Ones Matrix:
[1.0, 1.0]
[1.0, 1.0]
[1.0, 1.0]
Identity Matrix:
[1.0, 0.0, 0.0]
[0.0, 1.0, 0.0]
[0.0, 0.0, 1.0]


In [124]:
##### Part 6: Demonstrating Basic Properties of Matrices #####

print("========== TESTING PART 6 FUNCTIONS ==========")

def demonstrate_matrix_properties():
    A = Matrix(values=[[1, 2], [3, 4]])
    B = Matrix(values=[[5, 6], [7, 8]])
    C = Matrix(values=[[2, 0], [1, 3]])
    I = Matrix(values=[[1, 0], [0, 1]])
    
    print(f"\nMatrix A: \n{A}")
    print(f"\nMatrix B: \n{B}")
    print(f"\nMatrix C: \n{C}")
    print(f"\nMatrix I: \n{I}")
    
    # Associativity of Matrix Multiplication: (AB)C = A(BC)
    print("\n===== Testing if (AB)C = A(BC)=====")
    left = (A * B) * C
    right = A * (B * C)
    print("\n(AB)C:")
    print(left)
    print("\nA(BC):")
    print(right)
    print("\nAre they equal?", left == right)
    
    # Distributivity: A(B + C) = AB + AC
    print("\n===== Testing if A(B + C) = AB + AC =====")
    sum_matrix = B + C
    left = A * sum_matrix
    right = (A * B) + (A * C)
    print("\nA(B+C):")
    print(left)
    print("\nAB + AC:")
    print(right)
    print("\nAre they equal?", left == right)
    
    # Non-Commutativity: AB ≠ BA
    print("\n===== Testing if AB ≠ BA =====")
    AB = A * B
    BA = B * A
    print("\nAB:")
    print(AB)
    print("\nBA:")
    print(BA)
    print("\nAre they equal?", AB == BA)
    
    # Identity Matrix: AI = A
    print("\n===== Testing if AI = A =====")
    AI = A * I
    print("\nAI:")
    print(AI)
    print("\nIs AI equal to A?", AI == A)

demonstrate_matrix_properties()

========== TESTING PART 6 FUNCTIONS ==========

Matrix A: 
[1, 2]
[3, 4]

Matrix B: 
[5, 6]
[7, 8]

Matrix C: 
[2, 0]
[1, 3]

Matrix I: 
[1, 0]
[0, 1]

===== Testing if (AB)C = A(BC)=====

(AB)C:
[60, 66]
[136, 150]

A(BC):
[60, 66]
[136, 150]

Are they equal? True

===== Testing if A(B + C) = AB + AC =====

A(B+C):
[23, 28]
[53, 62]

AB + AC:
[23, 28]
[53, 62]

Are they equal? True

===== Testing if AB ≠ BA =====

AB:
[19, 22]
[43, 50]

BA:
[23, 34]
[31, 46]

Are they equal? False

===== Testing if AI = A =====

AI:
[1, 2]
[3, 4]

Is AI equal to A? True


# Quiz 2
Write a function make_deck that returns a list of all of the cards in a standard card deck. The return should be a list of tuples of pairs of suit and value. For example the 10 of Clubs would be ('Clubs', 10) and Queen of Hearts would be ('Hearts', 'Queen'). Recall that a deck has 52 cards, divided into 4 suits (Clubs, Diamonds, Hearts, and Spades), and that each suit has 13 cards: 2 to 10, Jack, Queen, King, and Ace. Summit your solution with Lab 5.

In [126]:
def make_deck():
    suits = ['Clubs', 'Diamonds', 'Hearts', 'Spades']
    values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'Jack', 'Queen', 'King', 'Ace']
    deck = [(suit, value) for suit in suits for value in values]
    return deck

# Example usage:
deck = make_deck()
for suit, value in deck:
    print(f"{value} of {suit}")

2 of Clubs
3 of Clubs
4 of Clubs
5 of Clubs
6 of Clubs
7 of Clubs
8 of Clubs
9 of Clubs
10 of Clubs
Jack of Clubs
Queen of Clubs
King of Clubs
Ace of Clubs
2 of Diamonds
3 of Diamonds
4 of Diamonds
5 of Diamonds
6 of Diamonds
7 of Diamonds
8 of Diamonds
9 of Diamonds
10 of Diamonds
Jack of Diamonds
Queen of Diamonds
King of Diamonds
Ace of Diamonds
2 of Hearts
3 of Hearts
4 of Hearts
5 of Hearts
6 of Hearts
7 of Hearts
8 of Hearts
9 of Hearts
10 of Hearts
Jack of Hearts
Queen of Hearts
King of Hearts
Ace of Hearts
2 of Spades
3 of Spades
4 of Spades
5 of Spades
6 of Spades
7 of Spades
8 of Spades
9 of Spades
10 of Spades
Jack of Spades
Queen of Spades
King of Spades
Ace of Spades
